[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

In [15]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [16]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [17]:
# Load JSON files and store the raw data in some variable. Edit as appropriate

# Define the filter for the PHE API call
filter = [
    'areaType=region',
    'areaName=London'
]

# Define the structure for the PHE API response
structure = {
    "date": "date",
    "dailyCases": "newCasesByPublishDate",
    "cumulativeCases": "cumCasesByPublishDate",
    "dailyDeaths": "newDeaths28DaysByPublishDate",
    "cumulativeDeaths": "cumDeaths28DaysByPublishDate"
}

# Create an instance of Cov19API with the specified filter and structure
api = Cov19API(filters=filter, structure=structure)

# Fetch the data from the API
timeseries=api.get_json()

# Create and store the data into canned.json
with open("canned.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)

# Read the data from canned.json and put it into a dictionary
jsondata={}

with open("canned.json", "rt") as INFILE:
    jsondata=json.load(INFILE)


In [18]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    # Extract the data and put it into a list
    dataList = rawdata['data']
    # Extract and sort the dates from the list
    dates=[dictionary['date'] for dictionary in dataList ]
    dates.sort()
    # Parse the start and end dates
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    # Create an empty dataframe with the date range and the following 4 columns
    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['dailyCases', 'cumulativeCases', 'dailyDeaths', 'cumulativeDeaths'])

    # Iterate through the data list
    for entry in dataList:
        # Extract and parse the date
        date=parse_date(entry['date'])
        # Iterate through each column
        for column in ['dailyCases', 'cumulativeCases', 'dailyDeaths', 'cumulativeDeaths']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(df.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                df.loc[date, column]=value
                
    # fill in any remaining "holes" due to missing dates
    df.fillna(0.0, inplace=True)
    # Return the dataframe
    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

In [19]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    try:
        # Define the filter for the PHE API call
        api_filter = [
        'areaType=region',
        'areaName=London'
        ]
        # Define the structure for the PHE API response
        api_structure = {
        "date": "date",
        "dailyCases": "newCasesByPublishDate",
        "cumulativeCases": "cumCasesByPublishDate",
        "dailyDeaths": "newDeaths28DaysByPublishDate",
        "cumulativeDeaths": "cumDeaths28DaysByPublishDate"
        }
        # Create an instance of Cov19API with the specified filter and structure
        api = Cov19API(filters=api_filter, structure=api_structure)
        # Fetch the data from the API
        api_data = api.get_json()
    # Error handling
    except(requests.exceptions.RequestException, JSONDecodeError) as e:
        print(f"Error when accessing API: {e}")
        api_data = {}
    # Return the data
    return api_data # return data read from the API

In [20]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    try:   
        """ Button callback - it must take the button as its parameter (unused in this case).
        Accesses API, wrangles data, updates global variable df used for plotting. """
        # Get fresh data from the API. If you have time, include some error handling
        # around this call.
        apidata=access_api()
        # wrangle the data and overwrite the dataframe for plotting
        global df
        df=wrangle_data(apidata)
        # the graph won't refresh until the user interacts with the widget.
        # this function simulates the interaction, see Graph and Analysis below.
        # The function needs to be adapted to your graph; you can omit this call
        # in the first instance
        refresh_graph()
        # after all is done, you can switch the icon on the button to a "check" sign
        # and optionally disable the button - it won't be needed again. If you are 
        # implementing error handling, you can use icons "unlink" or "times" and 
        apibutton.icon="check"
        #apibutton.disabled=True
    # change the button text to "Unavailable" when the api call fails.
    except(requests.exceptions.RequestException, JSONDecodeError):
        apibutton.icon="unlink"

# Create a button widget
apibutton=wdg.Button(
    description='REFRESH', 
    disabled=False,
    button_style='INFO',
    tooltip="Keep calm and carry on",
    icon='refresh'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets


# run all cells before clicking on this button

## The explanation of the dashboard
This dashboard allows you to visulise the Covid data such as dailyCases, cummulativeCases, dailyDeaths, cummulativeDeaths depending on the option that you can select from the dropdown 
list and you can also select the scale of the data (linear or log) depending on the option you select from the radio button. The mentioned data was fetched by calling the api call from 
PHE with the specified filter and structure.

In [23]:
# Define a graph plotting function
def plot_random_graph(opt, gscale):
    # Check the scale type based on the radio button
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    # Plot the selected column with the specified scale
    df[opt].plot(logy = logscale, color = 'red')
    # Put label on x and y axis
    plt.xlabel('date')
    plt.ylabel(opt)
    plt.show() # important! update won't work properly without this

# A dropdown widget to select a column
whichopt=wdg.Dropdown(
    options=['dailyCases', 'cumulativeCases', 'dailyDeaths', 'cumulativeDeaths'],
    value='dailyCases',
    disabled=False,
)
# A radio button widget to select the scale
scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)
# Create a VBox and a HBox as containers
vcontrols = wdg.VBox([whichopt, apibutton])
controls=wdg.HBox([vcontrols, scale])

# Define a function to redraw the graph when the data is updated
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=whichopt.value
    if current==whichopt.options[0]:
        other=whichopt.options[1]
    else:
        other=whichopt.options[0]
    whichopt.value=other # forces the redraw
    whichopt.value=current # now we can change it back
    
# connect the plotting function and the widgets    
graph=wdg.interactive_output(plot_random_graph, {'opt': whichopt, 'gscale':scale})

# actually displays the dashboard
display(controls, graph)

Output()

**Author and Copyright Notice** Remember that if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).